# Applying the EWAS_PRS to the UK BIOBANK DATA on the DNAnexus RAP



In [1]:
pip install -U scikit-learn

     |████████████████████████████████| 22.2 MB 31.0 MB/s eta 0:00:01
     |████████████████████████████████| 309 kB 114.1 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install -U statsmodels

     |████████████████████████████████| 9.5 MB 30.6 MB/s eta 0:00:01
     |████████████████████████████████| 233 kB 113.5 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
%%bash
wget https://s3.amazonaws.com/plink2-assets/plink2_linux_x86_64_20230607.zip
unzip plink2_linux_x86_64_20230607.zip

Archive:  plink2_linux_x86_64_20230607.zip
  inflating: plink2                  


--2023-06-29 12:33:00--  https://s3.amazonaws.com/plink2-assets/plink2_linux_x86_64_20230607.zip
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.204.221, 52.217.49.142, 52.216.48.168, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.204.221|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9212915 (8.8M) [application/zip]
Saving to: ‘plink2_linux_x86_64_20230607.zip’

     0K .......... .......... .......... .......... ..........  0%  659K 14s
    50K .......... .......... .......... .......... ..........  1%  663K 13s
   100K .......... .......... .......... .......... ..........  1%  663K 13s
   150K .......... .......... .......... .......... ..........  2%  142M 10s
   200K .......... .......... .......... .......... ..........  2%  123M 8s
   250K .......... .......... .......... .......... ..........  3%  668K 9s
   300K .......... .......... .......... .......... ..........  3%  189M 7s
   350K .......... .......... .......... ........

In [13]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
import warnings
import numpy as np
warnings.filterwarnings('ignore')
scaler = StandardScaler()
from subprocess import Popen, PIPE
import shlex
import re

def make_p(cancer, protein):
    file_name = protein+ ".sscore"
    cancer_file_name ="/mnt/project/Protein_work/pheno_files/"+ cancer + ".pheno"
    protein_scores = pd.read_csv(file_name, sep="\t")
    cancer_pheno = pd.read_csv(cancer_file_name, sep="\t")
    merged = protein_scores.merge(cancer_pheno, on="IID")
    small = merged[["IID","SCORE1_SUM", cancer]].copy()
    covar = pd.read_csv("/mnt/project/Protein_work/UKBB_covar_white.txt", sep="\t")
    dataset = small.merge(covar, on="IID")
    dataset['scaled_score'] = scaler.fit_transform(dataset[['SCORE1_SUM']])
    dataset["cc"] = dataset[cancer]
    formula = "cc ~ scaled_score +  SEX + age_recruitment + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10 "
    results = do_stuff(dataset,formula, protein, cancer)
    return results
    
def make_p_sex(cancer, protein):
    file_name = protein+ ".sscore"
    cancer_file_name ="/mnt/project/Protein_work/pheno_files/"+ cancer + ".pheno"
    protein_scores = pd.read_csv(file_name, sep="\t")
    cancer_pheno = pd.read_csv(cancer_file_name, sep="\t")
    merged = protein_scores.merge(cancer_pheno, on="IID")
    small = merged[["IID","SCORE1_SUM", cancer]].copy()
    covar = pd.read_csv("/mnt/project/Protein_work/UKBB_covar_white.txt", sep="\t")
    dataset = small.merge(covar, on="IID")
    dataset['scaled_score'] = scaler.fit_transform(dataset[['SCORE1_SUM']])
    dataset["cc"] = dataset[cancer]
    formula = "cc ~ scaled_score + age_recruitment + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10 "
    results = do_stuff(dataset,formula, protein, cancer)
    return results    

    
    
def do_stuff(test_set, formula1, protein, cancer):
    n = test_set[test_set["cc"] == 1 ].shape[0]
    n_cont = test_set[test_set["cc"] == 0 ].shape[0]
    #formula1 = "cc ~ scaled_score +  SEX + age_recruitment + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10 "
    logitfit = smf.logit(formula = formula1, data = test_set).fit(disp=0)
    params = logitfit.params
    conf = logitfit.conf_int()
    conf['Odds Ratio'] = params
    conf.columns = ['5%', '95%', 'Odds Ratio']
    moo = (round(np.exp(conf),3))
    OR = moo.iloc[1].values[2] 
    UPP = moo.iloc[1].values[1]
    LOW = moo.iloc[1].values[0]
    P = logitfit.pvalues[1]
    SE = logitfit.bse[1]
    Z = round(np.log(OR) / SE,2)
    log_results = pd.DataFrame( {'Protein': protein, 'Cancer': cancer,
     'OR': OR,
     'lower': LOW, 
     'upper' : UPP ,
     'p' : P , 
     'SE' : SE ,
     'Z' : Z, 
    }, index=[0])
    ### converting to str for plots
    log_results["n"] = str(n)
    log_results["n_cont"] = str(n_cont)
    log_results["P_val"] = log_results["p"].apply(lambda x:  '%0.2e' % x) 
   
    return log_results

def execute(cmd):
    """ to run commands on the shell and pipe out the display """
    plink_cmd_run = shlex.split(cmd)
    p = Popen(plink_cmd_run, shell=False, stdout=PIPE , stderr=PIPE)
    out, err = p.communicate()
    return (out)

def run_plink(protein):
    plink_cmd = ("./plink2 --pfile /mnt/project/Protein_work/ewas_pqtl --score /mnt/project/Protein_work/normal_clump_weights/"+protein+".clumpbyp.weights 1 2 3 list-variants  ignore-dup-ids cols=+scoresums --out " + protein)
    p = execute(plink_cmd)



In [16]:
%%bash 
ls /mnt/project/Protein_work/*.weights | cut -f 1 -d "." | cut -f 5 -d "/" > proteins.txt

In [8]:
proteins_df = pd.read_csv("get_scores.txt", header=None)
proteins_df.columns = ["ID"]

,ID
0,WAS
1,TPP1


In [9]:
%%time
for i in proteins_df.ID: 
    run_plink(i)

CPU times: user 44.4 ms, sys: 136 ms, total: 180 ms
Wall time: 2min 5s


In [10]:
tests = pd.read_csv("protein_outcomes.txt",sep="\t")

,Protein,cancer
0,WAS,bladder_inc
1,TPP1,headneck_inc
2,AGRN,kidneywho_inc


In [17]:
%%time
for i in tests.Protein: 
    run_plink(i)

CPU times: user 75.9 ms, sys: 308 ms, total: 384 ms
Wall time: 3min 24s


In [18]:
%%time

results = pd.DataFrame()

for index, row in tests.iterrows():
    protein = row['Protein']
    cancer =  row['cancer'] 
    try:
        temp = make_p(cancer, protein)
        results = results.append(temp)
        print (temp)
    except: 
        print ("trouble with " + protein + " " + cancer )
results
    

  Protein       Cancer    OR  lower  upper         p        SE     Z    n  \
0     WAS  bladder_inc  0.95  0.887  1.017  0.137913  0.034897 -1.47  835   

   n_cont     P_val  
0  335988  1.38e-01  
  Protein        Cancer     OR  lower  upper         p        SE    Z    n  \
0    TPP1  headneck_inc  0.966  0.902  1.033  0.311008  0.034568 -1.0  847   

   n_cont     P_val  
0  335976  3.11e-01  
  Protein         Cancer    OR  lower  upper         p        SE     Z     n  \
0    AGRN  kidneywho_inc  1.03   0.97  1.093  0.340092  0.030522  0.97  1082   

   n_cont     P_val  
0  335741  3.40e-01  
  Protein         Cancer     OR  lower  upper         p       SE     Z     n  \
0    CA12  kidneywho_inc  1.024  0.962  1.089  0.458833  0.03147  0.75  1082   

   n_cont     P_val  
0  335741  4.59e-01  
  Protein         Cancer     OR  lower  upper         p        SE     Z     n  \
0  CD300C  kidneywho_inc  0.967   0.91  1.027  0.279704  0.030852 -1.09  1082   

   n_cont     P_val  
0  33

,Protein,Cancer,OR,lower,upper,p,SE,Z,n,n_cont,P_val
0,WAS,bladder_inc,0.950,0.887,1.017,0.137913,0.034897,-1.47,835,335988,1.38e-01
0,TPP1,headneck_inc,0.966,0.902,1.033,0.311008,0.034568,-1.00,847,335976,3.11e-01
0,AGRN,kidneywho_inc,1.030,0.970,1.093,0.340092,0.030522,0.97,1082,335741,3.40e-01
0,CA12,kidneywho_inc,1.024,0.962,1.089,0.458833,0.031470,0.75,1082,335741,4.59e-01
0,CD300C,kidneywho_inc,0.967,0.910,1.027,0.279704,0.030852,-1.09,1082,335741,2.80e-01
...,...,...,...,...,...,...,...,...,...,...,...
0,CDH6,kidneywho_inc,1.041,0.982,1.105,0.179468,0.030168,1.33,1082,335741,1.79e-01
0,CDCP1,lungscc_inc,0.979,0.901,1.064,0.619913,0.042484,-0.50,555,336268,6.20e-01
0,BTN2A1,kidneywho_inc,0.945,0.888,1.006,0.075651,0.031857,-1.78,1082,335741,7.57e-02
0,BCAM,lungscc_inc,0.931,0.859,1.010,0.086057,0.041428,-1.73,555,336268,8.61e-02


In [20]:
results.to_csv("results_29_6_23.txt", sep="\t", index=None)

In [14]:
### Example code of running regressions without SEX in the covariates - these were ran individually
make_p_sex("uterine_inc", "KLK4")

,Protein,Cancer,OR,lower,upper,p,SE,Z,n,n_cont,P_val
0,KLK4,uterine_inc,1.006,0.947,1.069,0.845587,0.031031,0.19,1045,174435,8.46e-01


This next part merges the varaints that formed part of the score (Variants were missing from the processed exome-seq results) and adds this information to the single variant regressions

In [28]:
%%bash 
for i in $(ls /mnt/project/Protein_work/*.weights | cut -f 1 -d "." | cut -f 5 -d "/")
do
tail -n +2 /mnt/project/Protein_work/$i.weights | awk -v moo=$i ' { print $1"_"moo, $2 , $3 }' 
done > weights.all.txt

In [33]:
weights = pd.read_csv("weights.all.txt", delimiter=r"\s+", header=None )
weights.columns = ["key", "PRS_A1", "PRS_weight"]
scored_weights = pd.read_csv("all.txt", delimiter=r"\s+", header=None)
scored_weights.columns = ["key"] 
merged = weights.merge(scored_weights,on="key")
raw = pd.read_csv("single_variant.txt", sep="\t" ) 
merged = raw.merge(weights, on="key", how="left")
merged["PRS_weight"] = merged.PRS_weight.fillna("Variant not in score file") 
merged = merged.fillna("NA") 
merged.to_csv("Single_variants_with_PRS_weights.csv", index=None)

,key,PRS_A1,PRS_weight
0,1:151831737:G:A_ACE2,A,0.326750
1,12:20875274:G:C_ACE2,C,0.054925
2,14:103100498:C:G_ACE2,G,0.069187
3,14:103110107:C:G_ACE2,G,0.074866
4,14:24414681:G:A_ACE2,A,0.056008
...,...,...,...
12701,5:132486380:A:G_XCL1,A,0.050925
12702,7:95411704:G:C_XCL1,G,0.054794
12703,1:15916125:C:A_ZBTB17,C,0.065796
12704,10:113588287:G:A_ZBTB17,G,0.142864
